# Examples

This notebook uses PolicyEngine to analyse various UK policy reforms (the US interface is the same).

## Initialising

In [1]:
from policyengine import PolicyEngineUK

uk = PolicyEngineUK()

## Parametric reforms

In [2]:
# The best way to specify parametric reforms is to use the PolicyEngine parameter names.

baseline, reformed = uk.create_microsimulations(dict(basic_rate=0.21))

(
    reformed.calc("income_tax").sum() / 1e9
    - baseline.calc("income_tax").sum() / 1e9
)

4.759837825716403

## Custom reforms

In [3]:
# But you can also use the OpenFisca API (see the official documentation) to apply reforms.

from openfisca_core.model_api import *


class abolish_income_tax(Reform):
    def apply(self):
        self.neutralize_variable("income_tax")


baseline, reformed = uk.create_microsimulations(
    dict(reform=abolish_income_tax)
)

reformed.calc("income_tax").sum() / 1e9  # Should equal 0

0.0

In [7]:
# If you want to use inheritance:

from openfisca_uk import CountryTaxBenefitSystem

variables = CountryTaxBenefitSystem().variables


class income_tax(type(variables["income_tax"])):
    def formula(person, period, parameters):
        return (
            type(variables["income_tax"]).formula(person, period, parameters)
            + 1
        )


class add_1_pound_to_income_tax(Reform):
    def apply(self):
        self.update_variable(income_tax)


baseline, reformed = uk.create_microsimulations(
    dict(reform=add_1_pound_to_income_tax)
)

(
    reformed.calc("income_tax").sum() / 1e6
    - baseline.calc("income_tax").sum() / 1e6
)  # Should equal about 65

64.32629530166741